<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**<font size=6>💎Diamonds</font>**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json file is stored in Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 21jan25-diamonds         # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 21jan25-diamonds
Using competition: 21jan25-diamonds
  teamId  teamName                       submissionDate       score            
--------  -----------------------------  -------------------  ---------------  
13244663  1 Oluwatobi_Daniel_David       2025-02-02 19:23:21  575.1743229999   
13256813  12_Nainika_Arvin_Jenelle       2025-02-02 02:17:51  596.7469104999   
13271090  💎5 Sichao Liu and Yang Zhang   2025-02-03 00:08:33  611.5747210000   
13244737  💎 10                           2025-02-02 01:35:57  621.1262089999   
13256842  3 Shuai & Casimir              2025-02-03 02:19:07  629.7049565000   
13275591  2 Kumar Ray Josh               2025-02-03 00:05:07  633.0236220000   
13203520  💎 4_JacobMarx_ChipHenderson    2025-02-03 01:19:08  638.3123025000   
13244767  Team-9 💎                       2025-02-03 01:44:58  663.4061924716   
13240845  7_Austin_Madihah               2025-02-02 22:15:39 

In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals
os.environ['TF_DETERMINISTIC_OPS'] = '1'; os.environ['TF_CUDNN_DETERMINISTIC'] = '1'; # allows seeding RNG on GPU

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)
tDIR, sDIR = 'trainXY/', 'testY/'

CPU times: user 386 ms, sys: 829 µs, total: 387 ms
Wall time: 417 ms


In [ ]:
df = pd.read_csv('XY_diamonds.csv'); df

,carat,depth,table,x,y,z,cut,color,clarity,price
0,0.35,67.2,57.1,4.64,4.69,2.87,I,G,VS1,NaN
1,1.64,67.3,60.7,7.84,7.82,4.94,V,E,SI1,NaN
...,...,...,...,...,...,...,...,...,...,...
199998,0.48,68.7,53.0,4.47,4.45,2.72,I,G,IF,1474.0
199999,0.48,65.0,64.7,5.60,5.62,3.44,I,H,VS2,765.0


In [ ]:
df.price.info()

<class 'pandas.core.series.Series'>
RangeIndex: 200000 entries, 0 to 199999
Series name: price
Non-Null Count   Dtype  
--------------   -----  
160000 non-null  float64
dtypes: float64(1)
memory usage: 1.5 MB


In [ ]:
vX = df.query('price!=price').drop('price', axis=1)  # slice a test sample, price!=price is True for NaN values
tXY = df.query('price==price')                       # slice a training sample, price==price is True for non-NaN values
tX, tY = tXY.drop('price', axis=1), tXY.price        # split into training I/O

In [ ]:
def ScatterCorrHist(df):
  def corrdot(*args, **kwargs):
    # credit: https://stackoverflow.com/questions/48139899
    corr_r = args[0].corr(args[1], 'pearson')
    corr_text = f"{corr_r:2.2f}".replace("0.", ".")
    ax = plt.gca();
    ax.set_axis_off();
    msz = abs(corr_r) * 5000   # marker size
    fsz = abs(corr_r) * 40 + 5 # font size
    ax.scatter([.5], [.5], msz, [corr_r], alpha=0.5, cmap='coolwarm', vmin=-1, vmax=1, transform=ax.transAxes)
    ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction", ha='center', va='center', fontsize=fsz)

  sns.set(style='white', font_scale=.8);
  g = sns.PairGrid(df, aspect=1, diag_sharey=False);
  g.fig.set_size_inches(20,10)
  g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color':'red'}, scatter_kws={'s':1});
  g.map_diag(sns.histplot, kde_kws={'color':'black'});
  g.map_upper(corrdot);
  g.fig.suptitle("Scatter plot, Correlations and histograms on diagonal", y=1);
  _ = plt.subplots_adjust(hspace=0.02, wspace=0.02);
  _ = plt.show();

# ScatterCorrHist(tXY.head(200))  # takes a minute time to run

In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=green size=40>

<strong><font color=green size=5>⏳Timed Green Playground (TGP): Your ideas, code, documentation, and timer START HERE!</font></strong>

<font color=green>Students: Keep all your definitions, code, documentation in <b>TGP</b>. Modifying any code outside of TGP incurs penalties.

<font color=green><h3><b>$\alpha$. Build polynomial features</b><h3>






## Splitting data

We split our data taking care to ensure that we had a representative sample from the long tail of less-likely values that we can see in the distribution of training prices from below:

<img src='https://drive.google.com/uc?id=1v6Ele7a_JWjb8e5XPDlb08_75is7QiJH'>

By stratifying our samples in our train/test split, we achieved a better MAE, [see our writeup](#scrollTo=39zfwMOXwHAD).


In [ ]:
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  KBinsDiscretizer
from sklearn.model_selection import StratifiedShuffleSplit

# We have a continuous dependent variable and want to do stratified k-fold sampling so we can hopefully
# attend to the long tail of prices
# https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization.html
binning_transformer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
price_binned = binning_transformer.fit_transform(tY.values.reshape(-1, 1)).flatten()
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in stratified_split.split(tX, price_binned):
    X_train, X_test = tX.iloc[train_index], tX.iloc[test_index]
    y_train, y_test = tY.iloc[train_index], tY.iloc[test_index]

## Engineering features

In order to capture potential nonlinearities between the different features, we decided to poke around and try some different combined features through trial and error. More information can be found in [our ideas writeup](#scrollTo=39zfwMOXwHAD).

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin


def create_engineered_features(X: pd.DataFrame):
    X = X.copy()  # Avoid modifying the original data

    # Create new features
    X['volume'] = X['x'] * X['y'] * X['z']
    X['carat_volume'] = X['carat'] * X['volume']

    # Ratios
    X['table_to_carat_ratio'] = X['table'] / X['carat']
    X['depth_to_carat_ratio'] = X['depth'] / X['carat']

    return X

class FeatureEngineering(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = create_engineered_features(X)
        return X

## Clipping predictions

We noticed that the distribution of prices in the training and test sets start at a certain non-zero value, and end at a certain value. However, our initial models were predicting values that were not represented in the train/test sets, so we thought a naive (but effective) thing to do would be to clip values.

We ended up just clipping values between the lowest and highest price present in the training set.

Through this, we discovered that the regressor is an estimator (which is different than a transformer) and therefore doesn't need to return itself—hence the need to create a custom regressor (that contains the estimator that we want) instead of tacking on some sort of clipping transformer to the end of our pipeline.

In [ ]:
from dataclasses import dataclass

from sklearn.base import BaseEstimator, RegressorMixin

@dataclass
class ClippingRegressor(BaseEstimator, RegressorMixin):
    """Regressor that clips predictions between a min_val and max_val."""
    base_estimator: BaseEstimator
    min_val: float = 0.0
    max_val: float = 1.0

    def fit(self, X, y):
        self.base_estimator.fit(X, y)
        return self

    def predict(self, X):
        return np.clip(self.base_estimator.predict(X), self.min_val, self.max_val)

## Helper functions

We thought it was nice to have a way to print out not only the mean score and standard deviation of the best parameters found via grid search, but also of all the others.

In [ ]:
# Helper functions below

def _print_grid_search(grid_search):
    """Prints grid search results: best score, best params, and all scores."""
    idx = grid_search.best_index_
    score, std, params = (
        # Note: score inverted since it reports a negative value
        -grid_search.cv_results_["mean_test_score"][idx],
        grid_search.cv_results_["std_test_score"][idx], grid_search.cv_results_["params"][idx])

    print(f"Best Score: {score:.4f} ± {std:.4f}")
    print(f"Best Params: {params}")
    print("-----------")

    # Invert score since it reports a negative value
    means = -grid_search.cv_results_["mean_test_score"]
    stds = grid_search.cv_results_["std_test_score"]
    params = grid_search.cv_results_["params"]

    for mean, std, param in zip(means, stds, params):
        print(f"Score: {mean:.4f} ± {std:.4f}, Params: {params}")

<font color=green><h3><b>$\beta$. Fit the model to the training dataset</b><h3>

## Model pipeline

The model pipeline is defined below, steps comprising of:

1. Feature engineering: generating new features from our existing ones
2. Preprocessing: encoding the categorical variables either with ordinal or one-hot encoding
3. Polynomial: Creating polynomial features from our numerical features
4. Transforming: transforming our dependent variable using `log` and `exp` [3, pp. 75-78][4]
4. Regression: Using one of many different regression estimators (e.g., `Ridge`)
5. Clipping (clipping our output as described above)

In [ ]:
from sklearn.compose import ColumnTransformer, make_column_selector, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    FunctionTransformer, OrdinalEncoder, OneHotEncoder, PolynomialFeatures, MinMaxScaler, KBinsDiscretizer, StandardScaler,
)
from sklearn.linear_model import (
    LinearRegression, Ridge, RidgeCV,
    BayesianRidge, ElasticNetCV, LarsCV, LassoLarsCV, OrthogonalMatchingPursuitCV,
)
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
import numpy as np


# NOTE: Going off of https://www.brilliantearth.com/diamond/buying-guide/cut/ (which may be an assumption)
cut_categories = ["P", "F", "G", "V", "I"]
# NOTE: Going off of https://4cs.gia.edu/en-us/blog/diamond-color-chart-official-gia-color-scale/ (which may be an assumption)
color_categories = ["J", "I", "H", "G", "F", "E", "D"]
# NOTE: Going off of https://4cs.gia.edu/en-us/diamond-clarity/ (which may be an assumption)
clarity_categories = ["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"]

categorical_features = ["cut", "color", "clarity"]

ordinal_preprocessor = ColumnTransformer(
    transformers=[
        ("ordinal", OrdinalEncoder(categories=[
            cut_categories,
            color_categories,
            clarity_categories
        ]), categorical_features),
        ("num", StandardScaler(), make_column_selector(dtype_include='number')),
    ],
    remainder="passthrough"
)

onehot_preprocessor = ColumnTransformer(
    transformers=[
        ("onehot", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", StandardScaler(), make_column_selector(dtype_include='number')),
    ],
    remainder="passthrough"
)

log_transformer = FunctionTransformer(np.log, inverse_func=np.exp)

model_pipeline = Pipeline(
    steps=[
        ("feature_engineering", FeatureEngineering()),
        ("preprocessor", onehot_preprocessor),
        ("svd", TruncatedSVD(n_components=23)),
        ("poly", PolynomialFeatures(degree=3)),
        ("clipped_regressor", ClippingRegressor(
            base_estimator=TransformedTargetRegressor(
                regressor=Ridge(alpha=0.0001),
                transformer=log_transformer,
            ),
            min_val=y_train.min(),
            max_val=y_train.max())),
    ]
)

###############################
# HYPERPARAMETER OPTIMIZATION #
###############################
# param_grid = [{
#     "feature_engineering": [FeatureEngineering()],
#     # Ordinal has way fewer features
#     # TODO: If we use onehot, maybe we can PCA to dumb them down?
#     "preprocessor": [ordinal_preprocessor],
#     "preprocessor__num": [StandardScaler()],
#     # "clipped_regressor__base_estimator__alpha": [1],
#     "clipped_regressor__base_estimator__regressor": [Ridge()],
#     "clipped_regressor__base_estimator__regressor__alpha": [0.005, 0.01, 0.1],
# }]
#
# NOTE: Ensure that training cell below is commented out.
# m = GridSearchCV(
#     estimator=model_pipeline,
#     param_grid=param_grid,
#     cv=5,
#     scoring="neg_mean_absolute_error",
#     n_jobs=-1,
#     verbose=1
# )

# m.fit(X_train, y_train)

# _print_grid_search(m)
###############################
# HYPERPARAMETER OPTIMIZATION #
###############################

## Training

In [ ]:
m = model_pipeline.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=2.21188e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


<font color=green><h3><b>$\gamma$. Generate and post-process predictions.</font>

## Generating metrics

Generates MAE for both train and test sets.

In [ ]:
from sklearn.metrics import mean_absolute_error

# Predictions using the best model
train_Y_pred_best = m.predict(X_train)
test_Y_pred_best = m.predict(X_test)

# Calculate MAE for train and test
mae_train_best = mean_absolute_error(y_train, train_Y_pred_best)
mae_test_best = mean_absolute_error(y_test, test_Y_pred_best)

print(f"Train MAE: {mae_train_best}")
print(f"Test MAE: {mae_test_best}")

/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Train MAE: 558.152217662631
Test MAE: 565.9691243334823


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


## Export

Exports predictions from model `m`.

In [ ]:
pY = pd.DataFrame(m.predict(vX), index=range(1,len(vX)+1), columns=['price'])  # ensure that labels and observations are in corresponding order
pY = pY.clip(lower=10)  # ensures no negative prices

# Uncomment to visualize distribution
# _ = tY.hist(figsize=(40,3), bins=1000);  _ = plt.suptitle("Distribution of training prices");
# _ = pY.hist(figsize=(40,3), bins=1000);  _ = plt.suptitle("Distribution of predicted prices");

ToCSV(pY, 'TruncatedSVD23PolynominialDegree3Ridge0.0001')

/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


<font color=green><h3><b>$\epsilon$. Idea Documentation (JHU students only)</b></h3>
<details>
  <summary>Instructions</summary>
  <div>


1. **Audience**. Your peers who will learn from your Colab and ideas therein.
1. **Importance**. The ML/DL ideas are not entirely random, but are based on prior experience and systematized/organized experiments. We'd like students to share and learn from idea generation to idea experimentation process done in our class using tools learned thus far.
1. **Format**. Keep it concise/precise in consistent font/presentation. Include numbers/IDs to your References, such as [1] or [[Géron22]](https://scholar.google.com/scholar?cluster=498861685923226475), where these are defined in your References section below. This helps link your ideas/experiments to external ideas.
1. **Reproducibility**. Your description should contain reasonable details needed for reproducibility, i.e. describe the state of your modeling pipeline before the change is made, what is changed and how the idea was discovered, and what improvement it resulted in. Thus, peers can try this idea with an expectation of the value it brings. See examples below.
1. **Bonus** points for the exceptional/exemplary/educational documentation (see grading rubric).
****
1. **TODO**: Describe the key idea in your work in the following format (similar to a "micro publication"):
  1. **Title**. Give each idea a descriptive name (i.e. a micro abstract).
    1. Ex(ample). <i>"Thresholding carat feature outliers improves MAE by 3% on public LB"</i>
  1. **Idea Discovery**. What led you to this idea? Was it some [EDA](https://en.wikipedia.org/wiki/Exploratory_data_analysis), familiarity with this dataset or some of the features?
    1. Ex. <i>"We plotted all univariate distributions of variables and discovered that diamond carat had unreasonable (but rare) values below and above [0,10] interval, when plotted carat's histogram in the train and test sets, which contained 10 and 3 such outliers respectively. We decided to use 10 as a reasonable threshold because it is 99th percentile of carat values in the 20K baseline sample. See our histogram plot below [plot here]. "</i>
  1. **Finding's Importance**. Describe why you think the idea was important to proceed with.
    1. Ex. <i>"We use a linear model, the slope of which is sensitive to outliers on the periphery of the feature space domain. The fitted hyperplane slopes in the direction of the extreme training feature values thereby mapping a non-existent relation between carat size and diamond price, which is not expected to repeat in the test set. "</i>
  1. **Experiment Setup**.
  How did you set up experiments to test your idea? What resources were helpful? What metric did you select, why and what values did you observe?
    1. Ex. <i>"To alleviate the impact of the outlying feature values, we need to either remove observations with extreme values, or somehow cap them (to stay within the distribution of the other carat values) or use a model insensitive to outliers (such as robust regression). We learned 3 suitable methods for treating outliers in [ref]: ... [It'd be great to briefly describe each method] We tried each one on a Baseline model, while keeping the competition-required [MAE](https://en.wikipedia.org/wiki/Mean_absolute_error) metric. We tested each method locally on the seeded 50/50 split of the 20K training set sampled in baseline Colab."</i>
  1. **Results**. What was the result or metric improvement from implementing the experiment locally and/or on public LB?
    1. Ex. <i>"Baseline MAE was 539.1257546465 in public LB and 530 in local default experiment with 50/50 train-test split. When applied on the same-seed split, Methods 1,2,and 3 showed 1%, 2%, and 5% improvement on the test set. When uploaded to public LB, Method 3 showed a 3% improvement. So, we decided to keep method 3."</i>

</div> </details>
</font>


<font color=green><h4><b>Task 1. Preprocessing Ideas</b></h4>
<details>
  <summary>Instructions</summary>
  <div>Explain a <b>key idea</b> that helped in <b>preprocessing pipeline</b>. This may be about some feature engineering, tricky subsampling, clustering, dimension reduction, etc. Use the format in TODO specified above. Remember to provide citation references for the peers to read more into your work.
</div> </details>
</font>

1. **Title**: Stratified Sampling and Feature Engineering Reduces MAE by 5%
1. **Idea Discovery**:
A random train-test split could lead to an imbalanced price distribution between the training and testing sets, as we [mentioned above](#scrollTo=YR3IBcKevJrN) (and we noticed the long tail distribution of the training price data). Hence, stratifying our samples might improve generalization. Additionally, as hinted [in the Starter Ideas section](#scrollTo=q4QO-u3t8xAO), we thought it'd be interesting to engineer some compound features to see what effect there might be on the final result.
1. **Finding's Importance**:
  1. *Stratified Sampling*: Viewing the distribution of the predicted prices versus the training prices led us to believe that if we did random sampling to generate our test and train sets, we may not have enough of the long tail in either set. So we looked around and found that `scikit-learn` has exactly what we needed to ensure we have enough data from each bin [1][2].
  1. *Feature Engineering*: Being inspired by the starter ideas, we played around with different combinations of parameters that we thought might add some more information to our system (like a `table`-to-`carat` ratio).
1. **Experiment Setup**:
  1. *Train-test split*: We binned the train set (128,000) into five separate quantized bins using [`KBinsDiscretizer`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html), and we held out the remaining 20% of available data (32,000) as a test set. We then used `StratifiedShuffleSplit`[1] to sample into our train and test buckets. This is opposed to sampling using [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split).
  1. *Feature engineering*: We created a simple custom transformer (`FeatureEngineering`) to apply some different transformations, [such as `volume` and `table_to_carat_ratio`](#scrollTo=CDO-jQElsK_0).
  1. *Hyperparameter optimization*: We used [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) along with the requested Mean Absolute Error (MAE) metric to determine which combination of the above changes might yield the best MAE.
1. **Results**:
  1. Our best MAE value was found with the following parameters:
    1. *Feature Engineering*: Enabled
    1. *Stratified sampling*: Enabled
  1. Metrics
    1. Previous MAE that we were able to find (Ridge regressor): 644.31
    1. New MAE (training set): 609.50
    1. New MAE (test set): 613.76
  1. Therefore we were able to improve our MAE by 5% with our stratified sampling and feature engineering preprocessing (as measured with our local training and test sets; a commensurate change was noted on the public leaderboard).

<font color=green><h4><b>Task 2. Modeling Ideas</b></h4>
<details>
  <summary>Instructions</summary>
  <div>Explain a <b>key idea</b> that helped with <b>model selection</b> in the format specified above. This may include tuning model parameters (perhaps a grid search with specific parameter range) or some other experiments, search/choice of the suitable model, experiments with postprocessing of model predictions, etc. Use the format in TODO specified above. Remember to provide citation references for the peers to read more into your work.
</div> </details>
</font>

1. **Title**: Clipping Predictions
1. **Idea Discovery**: While exploring the dataset, we noticed that the price distribution in both the training and testing sets had non-zero values and capped at a certain maximum. However, our model was producing predictions outside the range of these values. To address this, we decided to clip the predicted values to ensure they fall within the range of the observed values in the dataset.
1. **Finding's Importance**: Since the performance of our model is evaluated based on the MAE of its predicted values, clipping any predictions that fall outside the expected range is a logical step. This adjustment helps minimize the gap between the predicted and actual values, improving overall accuracy.
1. **Experiment Setup**:
  1. We first ran the model without any clipping and reviewed the predictions. Next, we applied clipping to the output to assess the impact on performance.
  2. *Clipping Regressor*: As the regressor functions as an estimator, [mentioned above](#scrollTo=sSB2C-jx5xGa), we created a custom regressor incorporate clipping into the prediction process, ensuring that any values outside the desired range were clipped appropriately.
  3. We utilized the Mean Absolute Error (MAE) metric to determine the clipped models performance vs the unclipped performance.
1. **Results**:
  1. Metrics
    1. Previous MAE: 553.312
    2. New MAE: 552.627

1. **Title**: Optimizing Model Hyperparameters with GridSearchCV
1. **Idea Discovery**: While working on model training, we wanted to explore how different combinations of hyperparameters would affect model performance. Given that hyperparameters, such as the regularization strength and preprocessing methods, play a significant role in model accuracy, we decided to use GridSearchCV to systematically search for the best combination. This approach would help us optimize the model’s configuration to improve performance, as we noticed performance variability when changing hyperparameters manually.
1. **Finding's Importance**: Hyperparameter tuning is crucial in machine learning as it can significantly impact the model's ability to generalize. By using GridSearchCV, we were able to automate the process of testing various hyperparameter combinations, ensuring we identified the optimal setup.
1. **Experiment Setup**:
  1. We determined a range of potential values for each hyperparameter:
    1. Preprocessors: [OneHot encoder, Ordinal encoder]
    2. Scalar: [Standard Scaler, MinMax Scaler],
    3. Polynomial Features Degree: [1, 2, 3, 4, 5]
    4. Models: [Ridge, LinearRegression, Lasso, ElasticNet, HuberRegressor, BayesianRidge]
    5. Regressor Model alpha: [0.001, 0.005, 0.01, 0.1]  
  2. We than ran GridSearchCV to determine the best combinations of these parameters.
1. **Results**:
  1. Best Hyperparameters
    1. Preprocessors: OneHot Encoder
    2. Scalar: Standard Scaler,
    3. Polynomial Features Degree: 3
    4. Regressor Model: Ridge
    3. Regressor Model alpha: 0.0001   
  2. Metrics
    1. Previous MAE: 646.494
    2. New MAE: 568.595

<font color=green><h3><b>$\zeta$. References</b></h3>
<details>
  <summary>Instructions</summary>
  <div>

1. Cite your sources to help your peers learn from these (and to avoid plagiarism).
1. HOML textbook should be cited, since we used it in this week's learning.
1. Use Google Scholar to draw [APA](https://en.wikipedia.org/wiki/American_Psychological_Association) citation format for books and publications.
1. Cite [StackOverflow](https://stackoverflow.com/), YouTube videos, package docs, open-access textbooks/publicaitons and other meaningful internet resources that you used.
1. We may reward exceptional and meaningful citations (not just a list of [SKL](https://scikit-learn.org/stable/)/[TF](https://www.tensorflow.org/) manual pages and a list of articles.) For example, if you used an idea from a publication, indicate it in TGP with a number that corresponds to its reference in References.

</div> </details>
</font>

1. 3.1. Cross-validation: Evaluating estimator performance. (n.d.). Scikit-Learn. Retrieved February 1, 2025, from https://scikit-learn/stable/modules/cross_validation.html
1. “Using KBinsDiscretizer to Discretize Continuous Features.”. Retrieved February 1, 2025, from https://scikit-learn/stable/auto_examples/preprocessing/plot_discretization.html
1. Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 3rd Edition. (n.d.). Retrieved February 1, 2025, from https://learning.oreilly.com/library/view/hands-on-machine-learning/9781098125967/

<font size=5>⌛</font> <strong><font color=green size=5>Do not exceed competition's runtime limit! Do not write code outside TGP</font></strong>
<hr color=green size=40>

In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 62 sec > 60 sec limit!!!


<details>
  <summary><font size=5><b>💡Starter Ideas</b></font></summary>
  <div>
  
1. Tune model hyperparameters
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions ($x_i^a$, $x_ix_j$, ...) to identify new key features or their linear combinations. With too many features you can use a smaller subsample or reduce dimensionality of feature space using [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html), [tSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html), or [UMAP](https://umap-learn.readthedocs.io/en/latest/).
1. Do a thorough EDA and understand non-linear relation in I/O. Are any interactions more special than others (E.g. is diamond *volume* a better predictor?)
1. If you note a shift in trend for any predictors, try building a model for each trend.
1. Evaluate predictions and focus on poorly predicted "groups". Can they be identified and modeled separately?
1. Do scatter plots show piecewise linear shape? Can a separate linear model be used on each support of approximately linear X-Y relation?
1. How are categorical features treated by the SKLearn models? Is there a [better way](https://www.google.com/search?q=ways+to+encode+categorical+data) to encode these (perhaps, ordinal) features?
  1. E.g. you could replace codes (or groups of codes) with their frequencies, which may capture the implied "distance" or rarity between category levels.
  1. If encoding ordinal features with integers, should non-equidistant values be considered?
1. Learn about [modeling price of a diamond](https://www.google.com/search?q=machine+learning+model+price+diamond).
1. Try post-processing: shifting/scaling/transforming the distribution of predicted prices `pY` to match the distribution of training prices `tY`

</div> </details>